In [1]:
# Airline AI assistant with Gradio and tool calling

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import display, Markdown
import json

In [3]:
# Load environment variables in a file .env

load_dotenv()
openai_key=os.getenv("OPENAI_API_KEY")

# Check the key
if not openai_key:
    print("An API key was not found")
elif openai_key[:8]!="sk-proj-":
    print("An API key was found, but it dosen't start with sk-proj-")
elif openai_key.strip()!= openai_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("An API key was found and looks good")

An API key was found and looks good


In [4]:
# Create an instance of openai
openai = OpenAI()
openai_model = "gpt-4o-mini"

In [5]:
system_message = "You are a helpful assistant for an Airline called FlightAI." \
"Warmly welcome the user to FlightAI assistant." \
"Give short, courtese answers, no more than 1 sentence." \
"Always be accurate. If you don't know the answer, say so."


In [6]:
# Ticket prices
ticket_prices ={
    "london": "$799",
    "paris":"$899",
    "tokyo":"$1400",
    "berlin":"$499"

}

In [7]:
def get_ticket_price(destination_city):
    print (f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknow city price")
    return f"The price of a ticket to {destination_city} is {price}"

In [8]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                    "role": "tool",
                    "content": price_details,
                    "tool_call_id": tool_call.id
                })
    return responses

In [9]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=openai_model, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_responses = handle_tool_call(message)
        
        # Add the assistant's message with tool calls
        messages.append(message)
        
        # Add all tool responses
        for tool_response in tool_responses:
            messages.append(tool_response)
        
        # Get final response from the model after tool responses
        response = openai.chat.completions.create(model=openai_model, messages=messages)
    
    return response.choices[0].message.content

Tools

In [12]:
# get_ticket_price("abuja")

In [13]:
# Gradio implementation
gr.ChatInterface(fn=chat, title="FlightAI Assistant").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
